In [0]:
# !pip install --upgrade tensorflow -q
# import os
# os._exit(0)

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from IPython.display import display
import time

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

In [0]:
# !unzip -q ml-100k.zip
# !rm ml-100k.zip

In [0]:
#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

#Reading items file:
i_cols = ['movie_id', 'movie title' ,'release date','video release date', \
          'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', \
          'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', \
          'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

In [0]:
user, item, title = 'user_id', 'movie_id', 'movie title'

In [6]:
display(users); display(ratings); display(items);

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
rating_movie = ratings.merge(items[[item, title]])
rating_movie.head()

,user_id,movie_id,rating,unix_timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [8]:
id2movie, movie2id = {}, {}

n_users = users.shape[0]
n_items = items.shape[0]

ui_matrix = np.zeros((n_users, n_items), dtype=np.int8) # user-item matrix

for (_, ui, mi, rating, _, mt) in rating_movie.itertuples():
    id2movie[mi] = mt
    movie2id[mt] = mi
    ui_matrix[ui-1, mi-1] = rating

print(id2movie[1], movie2id['Toy Story (1995)'], sep='\n')

Toy Story (1995)
1


In [9]:
BUFFER_SIZE = len(rating_movie)
BATCH_SIZE = 128
STEPS_PER_EPOCH = len(rating_movie)//BATCH_SIZE

dataset = tf.data.Dataset.from_tensor_slices((rating_movie[[user, item]].values - 1, 
                                            rating_movie['rating'].values))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)

dataset

<BatchDataset shapes: ((None, 2), (None,)), types: (tf.int64, tf.int64)>

In [0]:
x, y = iter(dataset.take(STEPS_PER_EPOCH)).next()

### Matrix factorization

In [0]:
class Collab(tf.keras.Model):
    def __init__(self, n_user, n_item, n_dim=50, y_range=(0, 10)):
        super().__init__()
        self.user_w = tf.keras.layers.Embedding(n_user, n_dim)
        self.item_w = tf.keras.layers.Embedding(n_item, n_dim)
        self.user_b = tf.keras.layers.Embedding(n_user, 1)
        self.item_b = tf.keras.layers.Embedding(n_item, 1)
        self.y_range = y_range
  
    def call(self, user, item):
        x = self.user_w(user) * self.item_w(item)
        x = tf.reduce_sum(x, axis=1)
        x = x + tf.squeeze(self.user_b(user) + self.item_b(item))
        x = tf.sigmoid(x) * (self.y_range[1] - self.y_range[0]) + self.y_range[0]

        return x

model = Collab(n_user=n_users, n_item=n_items,
               n_dim=30, y_range=(0, 5.5))

optimizer = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.MeanSquaredError()

In [0]:
@tf.function
def train_step(user, item, rating):
    
    loss = 0

    with tf.GradientTape() as tape:
        output = model(user, item)
        loss += loss_func(rating, output)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

In [16]:
EPOCHS = 5

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (user_item, rating)) in enumerate(dataset.take(STEPS_PER_EPOCH)):
        user, item = zip(*user_item)
        user, item = np.array(user), np.array(item)

        batch_loss = train_step(user, item, rating)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / STEPS_PER_EPOCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.9896
Epoch 1 Batch 100 Loss 2.0457
Epoch 1 Batch 200 Loss 1.7100
Epoch 1 Batch 300 Loss 1.3871
Epoch 1 Batch 400 Loss 1.4977
Epoch 1 Batch 500 Loss 1.6953
Epoch 1 Batch 600 Loss 1.0341
Epoch 1 Batch 700 Loss 1.0061
Epoch 1 Loss 1.4228
Time taken for 1 epoch 20.552567720413208 sec

Epoch 2 Batch 0 Loss 0.8173
Epoch 2 Batch 100 Loss 1.0243
Epoch 2 Batch 200 Loss 1.1047
Epoch 2 Batch 300 Loss 0.7648
Epoch 2 Batch 400 Loss 0.6951
Epoch 2 Batch 500 Loss 0.9830
Epoch 2 Batch 600 Loss 0.8263
Epoch 2 Batch 700 Loss 0.8901
Epoch 2 Loss 0.9019
Time taken for 1 epoch 19.708974361419678 sec

Epoch 3 Batch 0 Loss 0.6832
Epoch 3 Batch 100 Loss 0.6745
Epoch 3 Batch 200 Loss 0.9148
Epoch 3 Batch 300 Loss 0.7248
Epoch 3 Batch 400 Loss 0.6812
Epoch 3 Batch 500 Loss 0.8936
Epoch 3 Batch 600 Loss 0.8984
Epoch 3 Batch 700 Loss 0.7183
Epoch 3 Loss 0.7948
Time taken for 1 epoch 19.63145399093628 sec

Epoch 4 Batch 0 Loss 0.6065
Epoch 4 Batch 100 Loss 0.6446
Epoch 4 Batch 200 Loss 0.762